<a href="https://colab.research.google.com/github/iued-uni-heidelberg/projTermiKnowledge/blob/main/projTermiKnowledge2021ExtractComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook pre-processes files in a "projCorpus" directory and extracts user comments from the webpage.

The process has been created for the articles on this website:


It can be adjusted to other articles with known format.

In [1]:
# download the archive file with a directory, where the html files are stored:
!wget https://heibox.uni-heidelberg.de/f/ac9dae5cf1d9453bb3d0/?dl=1

--2021-10-08 08:23:58--  https://heibox.uni-heidelberg.de/f/ac9dae5cf1d9453bb3d0/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/1d029934-f7a3-4b60-bc22-bf2a2576a1a1/projCorpus.zip [following]
--2021-10-08 08:23:59--  https://heibox.uni-heidelberg.de/seafhttp/files/1d029934-f7a3-4b60-bc22-bf2a2576a1a1/projCorpus.zip
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 1586160 (1.5M) [application/zip]
Saving to: ‘index.html?dl=1’

index.html?dl=1     100%[===================>]   1.51M  2.16MB/s    in 0.7s    

2021-10-08 08:23:59 (2.16 MB/s) - ‘index.html?dl=1’ saved [1586160/1586160]



In [2]:
# rename the file:
!mv index.html\?dl\=1 projCorpus.zip

In [3]:
# extract corpus
!unzip projCorpus.zip

Archive:  projCorpus.zip
   creating: projCorpus/
  inflating: projCorpus/Enter Air wysyła samolot po pasażerów. Komunikat MSZ_ Wszyscy polscy obywatele są bezpieczni.mhtml  
   creating: __MACOSX/
   creating: __MACOSX/projCorpus/
  inflating: __MACOSX/projCorpus/._Enter Air wysyła samolot po pasażerów. Komunikat MSZ_ Wszyscy polscy obywatele są bezpieczni.mhtml  
  inflating: projCorpus/Siatkarska układanka Ryszarda Czarneckiego. Wśród działaczy krąży nagranie - blog Salon24 news.mhtml  
  inflating: __MACOSX/projCorpus/._Siatkarska układanka Ryszarda Czarneckiego. Wśród działaczy krąży nagranie - blog Salon24 news.mhtml  


In [4]:
# optional: remove zip file -- to save space
!rm projCorpus.zip

creating a class for walking through directories and processing html files

In [8]:
# -*- coding: utf-8 -*-
# Python script to open each file, read json input and copy to one text file for subsequent processing
import os, re, sys


class clHtml2txt(object):
    '''
    @author Bogdan Babych, IÜD, Heidelberg University
    '''
    def __init__(self, SDirName = 'projCorpus', SOutFile = 'projCorpus-out.txt'):
        self.InputDir = SDirName
        self.OutputFile = SOutFile
        self.ID = 100000
        self.STag = 'doc'
        # calling a function for walking directories
        self.openDir(self.InputDir)
        return

    def openDir(self, path): # implementation of recursively openning directories from a given rule directory and reading each file recursively into a string

        FOut = open(self.OutputFile, 'w')

        for root,d_names,f_names in os.walk(path):
            i = 0
            for f in f_names:
                i+=1
                if i%1==0: print(str(i) + '. Processing: ' + f)
                fullpath = os.path.join(root, f)
                # print(fullpath)
                try:
                    FIn = open(fullpath,'r')
                    SIn = FIn.read()

                    SText2Write = self.procFile(SIn,f,i)
                    if SText2Write: FOut.write(SText2Write) # if the string is not empty then write to file
                    FIn.close()
                except:
                    print(f'file {f} cannot be read or processed')

        FOut.flush()
        FOut.close()

        return

    def procFile(self, SIn,SFNameIn,i):
        SText2Write = ''
        '''
        processing an individual file: finding comments and returning them as a string to write
        '''
        STagOpen = '<' + self.STag + ' id="' + self.STag + str(self.ID + i)  + '">\n'
        STagClose = '\n</' + self.STag + '>\n\n'
        SText4Corpus = self.getArticleNComments(SIn, SFNameIn)

        if SText4Corpus:
            return STagOpen + SText4Corpus + STagClose
        else:
            print('\tNo data read from: ' + SFNameIn)
            return None



    def getArticleNComments(self, SIn, SFNameIn):
        SIn = re.sub('=\n', '', SIn, flags = re.MULTILINE | re.DOTALL)
        SText4Corpus = ''

        LParagraphs = re.findall('<p class="art_paragraph">(.+)</p>', SIn, flags = re.MULTILINE | re.DOTALL)
        SParagraphs = '\n\n'.join(LParagraphs)
        SParagraphs = re.sub('<.+?>', '', SParagraphs, flags = re.MULTILINE)

        LComments4Corpus = re.findall('<div class=3D"commentsApp__commentBody">(.+)</div>', SIn, flags = re.MULTILINE | re.DOTALL)
        SComments4Corpus = '\n\n'.join(LComments4Corpus)
        SComments4Corpus = re.sub('<.+?>', '', SComments4Corpus, flags = re.MULTILINE)
        
        return SParagraphs, SComments4Corpus

In [7]:
OHtml2txt = clHtml2txt()

UnboundLocalError: ignored